<center> <h1> Restoration of underwater images </h1> <br> <h3> <br> 4MEO Computer vision 2</h3> </center>

# I. Introduction

### 1. Import libraries

In [4]:
import numpy as np
import scipy as sc
import cv2 as cv
import matplotlib.image as mpimg
import matplotlib.pylab as plt

# II. Compute inputs

## 1. Show function

In [5]:
def show(final):
    cv.imshow('White balanced', final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    for i in range (1,5):
        cv.waitKey(1)

## 2. White balance function definition 

In [6]:
def white_balance(img):
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

## Output

In [7]:
firstInput = cv.imread('data/test1.png')
final = np.hstack((firstInput, white_balance(firstInput)))
show(final)